# Introduzione a Redis

## Nozioni Base
Redis e' un in-memory open source DBMS, rilasciato sotto licenza BSD, che viene generalmente utilizzato anche come cache, message broker / pub-sub e per processare stream di dati. Redis dispone di data structures come strings, hashes, lists, sets, sorted sets con diverse queries, bitmaps, hyperloglogs, geospatial indexes, e streams. Redis ha un built-in replica system che permette di creare repliche del medesimo DB, Lua scripting, LRU eviction, transazioni e diversi livelli di on-disk persistence.

Redis può essere usato su quasi tutti i linguaggi di programmazione disponibili tramite framework implementati durante il corso degli anni.

## Chi usa Redis?
Redis e' ampliamente utilizzato, direttamente o indirettamente, da quasi ogni azienda sul mercato, ad esempio come cache di dati per database relazionali, come database non relazionale sfruttando le sue funzionalita' di Key-Value store ricevendo dati da stream esterni, e' utilizzato da grandi aziende come backend per le CDN (Content Distribution Network) come CloudFlare, ad oggi utilizzate on quasi qualsiasi infrastruttura di medio / alto livello, per accelerare la fornitura di contenuti ai browser e/o ai dispositivi mobile, e' utilizzato per processare stream di dati per AI e ML da aziende come ChatGPT, e via dicendo.

Un elenco non esaustivo di aziende che ad oggi sfruttano Redis, in aggiunta a quelle sopra citate, include:
- Microsoft
- Google
- Amazon
- Twitter
- Github
- Snapchat, notoriamente grande fan di Redis al punto di acquisire KeyDB, un competitor di Redis, per avere una soluzione in-house
- Stackoverflow

E altre ancora

## Installazione
### Windows
Redis non è supportato su Windows, ma si può installare sfruttando la WSL2 (Windows Subsystem for Linux).

### Linux
Avendo in commercio una gran quantità di distribuzioni Linux, ogni distribuzione ha una guida di riferimento su come installare il pacchetto Redis.
Su sistemi Debian/Ubuntu possiamo aggiungere il Repository APT Ufficiale `packages.redis.io` e installare successivamente il pacchetto:
```bash
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

sudo apt-get update
sudo apt-get install redis
```
Possiamo sfruttare questi comandi per installare la versione WSL anche su Windows.

Alternativamente possiamo sfruttare il pacchetto Redis disponibile sullo Snapcraft Store e installarlo su tutte le distribuzioni che lo supportano:

```bash
sudo snap install redis
```

### MacOS
Per installare il pacchetto Redis su dispositivi macOS, sarà necessario in prima battuta installare Brew, un package manager creato per il Sistema Operativo di Cupertino.
Una volta installato correttamente si dovrà semplicemente lanciare il comando:
```bash
brew install redis
```

Per avviare il servizio si dovrà usare:
```bash
redis-server
```

Per fermare il servizio andrà premuto `CTRL-C`

## Pro e Contro
PRO:

- Velocità: Redis è estremamente performante, in grado di gestire grandi quantità di dati in modo rapido e efficiente, grazie alla sua architettura in memoria.

- Scalabilità: Redis è in grado di scalare orizzontalmente, ovvero di aggiungere nodi al cluster per aumentare la capacità di gestione dei dati.

- API: Redis offre una vasta gamma di API per supportare la gestione dei dati, tra cui la memorizzazione di dati semplici, liste, set e mappe.

- Persistenza dei dati: Redis offre la possibilità di persistere i dati su disco, garantendo la durata dei dati in caso di malfunzionamenti del server.

- Architettura semplice: Redis è facile da utilizzare e da configurare, anche per gli utenti meno esperti.

CONTRO:

- Scalabilità: Anche se Redis è in grado di scalare orizzontalmente, aggiungendo nodi ai cluster, non sfrutta al massimo le capacità della singola macchina. Per questo sono state create versioni multi-thread ad esempio: [cachegrand.io](https://cachegrand.io/)

- Capacità limitata: Poiché Redis utilizza la memoria per archiviare i dati, la capacità di archiviazione è limitata dalla quantità di memoria disponibile sul server.

- Nessun supporto per SQL: Redis non supporta SQL, quindi non è adatto per applicazioni che richiedono complesse operazioni di query, sotto viene presentato un modulo esterno chiamato RediSearch che ci permetterà di avere delle simil queries.

- Nessun supporto per transazioni complesse: Redis non supporta le transazioni complesse, il che lo rende meno adatto per le applicazioni che richiedono operazioni di transazione complesse e transazioni ACID, le uniche transazioni supportate sono semplici sequenze di comandi.

## Librerie del DBMS
Quasi tutti i linguaggi di programmazione hanno disponibile un porting per connettersi al servizio Redis, lavorando con Python andremo nel dettaglio di quest'ultima.
### redis-py
**Installazione**

Per installare redis-py:
```bash
pip install redis-py
```

Per migliorare le performance si può utilizzare installa il pacchetto con il supporto ad `hiredis`, che offre un Response Parser compilato, che per molte situazioni richiede 0 modifiche.

```bash
pip install redis[hiredis]
```

Link alla documentazione di [redis-py](https://redis-py.readthedocs.io/en/stable/)

Installazione di redis-py su Jupiter:

In [1]:
%pip install redis

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Creazione e Connessione al DB
Per facilitare lo sviluppo su più sistemi operativi, questo progetto dispone di un docker-compose.yml file, contenente il necessario per avviare un'istanza redis con due User Interfaces, quali `redis-cli` e `RedisInsight`.

Per avviare il servizio si può lanciare il comando:
```bash
make start
```

### Connessione al DB tramite redis-py

In [2]:
import redis
from redis.commands.json.path import Path
from redis.commands.search.field import TextField, NumericField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

## Operazioni CRUD
In Redis, le operazioni CRUD (Create, Read, Update, Delete) sono eseguite tramite comandi specifici per la manipolazione dei dati.

Ecco una panoramica delle operazioni CRUD in Redis:

- CREATE: L'operazione di creazione di un nuovo dato in Redis viene eseguita utilizzando il comando SET. Ad esempio, per creare un nuovo valore "valore1" associato alla chiave "chiave1", si può utilizzare il comando SET chiave1 valore1.

In [3]:
r.set('nome', 'Marco')

True

- READ: Per leggere i dati associati a una chiave, si può utilizzare il comando GET. Ad esempio, per leggere il valore associato alla chiave "chiave1", si può utilizzare il comando GET chiave1.

In [4]:
r.get('nome')

'Marco'

- UPDATE: L'operazione di aggiornamento di un dato esistente in Redis viene eseguita utilizzando il comando SET. Ad esempio, per aggiornare il valore associato alla chiave "chiave1" con il nuovo valore "valore2", si può utilizzare il comando SET chiave1 valore2.

In [5]:
r.set('nome', 'Francesco')

True


- DELETE: Per eliminare una chiave e il valore associato, si può utilizzare il comando DEL. Ad esempio, per eliminare la chiave "chiave1" e il valore associato ad essa, si può utilizzare il comando DEL chiave1.

In [6]:
r.delete('nome')

1

Inoltre, Redis offre anche altri comandi per manipolare i dati in modo più specifico, come ad esempio il comando INCR per incrementare il valore numerico di una chiave, il comando LPUSH per aggiungere un valore a una lista, e il comando SADD per aggiungere un elemento a un set.

## Comparazione con i DBMS visti a lezione

Redis è un DB Key-Value che offre velocità, affidabilità e prestazioni, posto a confronto con altri DB abbiamo alcune differenze:

- Nel DB in analisi la differenza principale nel salvataggio dei dati in memoria è che questi non vengono salvati nella memoria secondaria ma bensì nella memoria primaria rendendo le operazioni di lettura e di scrittura molto veloci 
- La struttura delle query è molto semplice e varia in base alla versione di Redis che si utilizza: bisogna definire la funzione da eseguire e successivamente i parametri.

In [7]:
r.rpush("mylist", 1, 2, 3, 4, 5, "hello world")
# RPUSH mylist 1 2 3 4 5 "hello world"

6

&emsp;&emsp;&ensp;La funzione sopra indicata permette di eseguire una push in una lista aggiungendo sei elementi, per poter vedere gli elementi all'interno di una lista basta semplicemente eseguire:

In [8]:
r.lrange("mylist", 0, -1)

['1', '2', '3', '4', '5', 'hello world']

Da questi esempi (non complessi) si può notare la semplicità e la facilità di uso di questo DB. 
In poco tempo con il supporto della documentazione si può imparare e si può utilizzare al meglio questo strumento.
Infine la documentazione presente sul sito di <link href="redis.io">redis.io</link> è disponibile solamente in inglese, permette di seguire l'utente dall'installazione all'utilizzo completo del Database e consente di imparare tutte le nozioni base (e anche non) in poco tempo in una guida descritta semplicemente e passo passo. 

## Esempi di query (Complesse)
E' importante notare che Redis non è un database relazionale tradizionale con un linguaggio di query completo come SQL. Redis è un database di strutture dati in memoria che offre un set di comandi per la manipolazione di essi.

Nel contesto di Redis, le query complesse sono solitamente realizzate concatenando i comandi disponibili nel Redis Command Language. Vengono forniti un insieme di comandi predefiniti per interagire con Redis, come SET, GET, HSET, HGET, ecc., che possono essere utilizzati per leggere, scrivere o manipolare i dati all'interno delle strutture di dati come stringhe, liste, set, hash, ecc.

Redis fornisce inoltre un sistema di scripting integrato, tramite il linguaggio Lua, che permette di eseguire più comandi e creare logiche complesse.

Ai fini del progetto verranno introdotto due moduli esterni, ma disponibili all'interno della docker image <code>redis-stack</code> denominati [RediSearch](https://redis.io/docs/stack/search) e [RedisJSON](https://redis.io/docs/stack/json/), redis-stack contiene altri moduli per il supporto a Grafi e TimeSeries.

## Introduzione ai Moduli Esterni
RediSearch è un modulo open source di Redis che abilita all'uso di query, secondary indexing e full-text search sul DB. Queste funzionalità permetto queries su più fields, aggregation, exact phrase matching, come visto anche all'interno di ElasticSearch.
Tramite questo modulo possiamo dunque simulare all'interno di redis un comportamento simile a quello di ES.

Il modulo RedisJSON abilita al supporto per i JSON, permettendo di salvare, aggiornare e recuperare dei JSON values all'interno del DB, in modo similare ai tipi già coperti da Redis. RedisJSON funziona autoamaticamente con RediSearch permettendo di creare indici e effettuare query su documenti JSON.

Per illustrare al meglio le qualità di Redis, abbiamo deciso di utilizzare i dati JSON utilizzati durante le lezioni dai professori, al fine di confrontarne la velocità di esecuzione e la complessità nella creazione delle query.

Il codice seguente rappresenta la definizione dello schema del JSON nel quale vengono specificati gli attributi di ciascun campo. Per definire i valori nidificati all'interno di JSON bisogna, ad esempio ,  scrivere $.name.english per definire"english" dentro a "name".
All'interno del file JSON è presente il campo "Type" che è un array di stringhe, per poterlo salvare utilizziamo TextField e i dati presenti saranno salvati come un array.

In [9]:
## Creiamo lo schema per l'indice
schema = (NumericField('$.id', as_name='id'),
          TextField('$.name.english', as_name='english'),
          TextField('$.name.japanese', as_name='japanese'),
          TextField('$.name.chinese', as_name='chinese'),
          TextField('$.name.french', as_name='french'),
          TextField('$.type', as_name='type'),
          NumericField('$.base.HP', as_name='hp'),
          NumericField('$.base.Attack', as_name='att'),
          NumericField('$.base.Defense', as_name='def'),
          NumericField('$.base.Sp. Attack', as_name='spatt'),
          NumericField('$.base.Sp. Defense', as_name='spdef'),
          NumericField('$.base.Speed', as_name='spd'))

## Creiamo l'indice se non esiste
try:
    r.ft("pokemon").create_index(schema, definition=IndexDefinition(
        prefix=['pokemon:'], index_type=IndexType.JSON))
except:
    # passa oltre se esiste già
    pass

from redis.commands.search.query import NumericFilter, Query
import redis.commands.search.aggregation as aggregations
import redis.commands.search.reducers as reducers
import time

Ai fini della leggibilità sono stati ridotti a 3 il numero di risultati visualizzabili.

- Cerca tutti i pokemon che contengono Pika nel loro nome:

In [10]:
r.ft("pokemon").search("Pika*").docs

[Document {'id': 'pokemon:25', 'payload': None, 'json': '{"id":25,"name":{"english":"Pikachu","japanese":"ピカチュウ","chinese":"皮卡丘","french":"Pikachu"},"type":["Electric"],"base":{"HP":35,"Attack":55,"Defense":40,"Sp. Attack":50,"Sp. Defense":50,"Speed":90}}'}]

La query Redis sopra mostrata permette la ricerca di tutti i Pokemon che iniziano con "Pika".
Utilizzando la wildcard '*' alla fine del termine di ricerca "Pika" la query restituirà tutti i documenti che iniziano con "Pika".

- Cerca tutti i pokemon di tipo Fuoco (contenendo anche i tipi misti come Fuoco-Volante)

In [11]:
query = Query("@type:Fire").paging(0, 3)
r.ft("pokemon").search(query).docs

[Document {'id': 'pokemon:4', 'payload': None, 'json': '{"id":4,"name":{"english":"Charmander","japanese":"ヒトカゲ","chinese":"小火龙","french":"Salamèche"},"type":["Fire"],"base":{"HP":39,"Attack":52,"Defense":43,"Sp. Attack":60,"Sp. Defense":50,"Speed":65}}'},
 Document {'id': 'pokemon:5', 'payload': None, 'json': '{"id":5,"name":{"english":"Charmeleon","japanese":"リザード","chinese":"火恐龙","french":"Reptincel"},"type":["Fire"],"base":{"HP":58,"Attack":64,"Defense":58,"Sp. Attack":80,"Sp. Defense":65,"Speed":80}}'},
 Document {'id': 'pokemon:6', 'payload': None, 'json': '{"id":6,"name":{"english":"Charizard","japanese":"リザードン","chinese":"喷火龙","french":"Dracaufeu"},"type":["Fire","Flying"],"base":{"HP":78,"Attack":84,"Defense":78,"Sp. Attack":109,"Sp. Defense":85,"Speed":100}}'}]

La query per la ricerca di tutti i pokemon fuoco effettua una range query ritornando qualsiasi pokemon abbia il tipo "Fire" all'interno del campo "Type".

- Cerca i primi 10 pokemon che hanno un attacco compreso fra 50 e 100 (estremi inclusi)

In [12]:
query = Query("@att:[50 100]").return_fields("english", "att").paging(0, 3)
r.ft("pokemon").search(query).docs

[Document {'id': 'pokemon:2', 'payload': None, 'english': 'Ivysaur', 'att': '62'},
 Document {'id': 'pokemon:3', 'payload': None, 'english': 'Venusaur', 'att': '82'},
 Document {'id': 'pokemon:4', 'payload': None, 'english': 'Charmander', 'att': '52'}]

La query definita consente di trovare i primi 10 pokemon (utilizzando il metodo .pagin(0, 10)) che hanno un attacco compreso fra 50 e 100 (Query("@att:[50 100]")).
Il metodo return:fields("english", "att") è stato aggiunto per semplificare la lettura del risultato.

- Trovare tutti i pokémon con difesa minore di 90 o di tipo “Fire”. Proiettare solo l’attacco, il nome in inglese e i tipi. Limitare i risultati da visualizzare a 10.

In [13]:
query = Query("@def:[0 90] | @type:Fire").return_fields("english", "type", "att").paging(0, 3)
r.ft("pokemon").search(query).docs

[Document {'id': 'pokemon:4', 'payload': None, 'english': 'Charmander', 'type': '["Fire"]', 'att': '52'},
 Document {'id': 'pokemon:5', 'payload': None, 'english': 'Charmeleon', 'type': '["Fire"]', 'att': '64'},
 Document {'id': 'pokemon:6', 'payload': None, 'english': 'Charizard', 'type': '["Fire","Flying"]', 'att': '84'}]

Questa query ritorna tutti i pokemon che hanno difesa compresa tra 0 e 90 O (inserendo l'operatore logico | (OR) e scrivendo all'interno dei valori del metodo Query due condizioni) che sono fuoco.
I campi dei risultati vengono limitati a "english", "type" e "att" e vengono mostrati i primi 10 elementi.

Si può evincere nei risultati di questa query che i documenti non sono ordinati e sono tutti pokemon di tipo fuoco:
- I risultati non sono ordinati poiché i documenti restituiti da una query non viene garantito che siano ordinati nell'ordine in cui sono stati inseriti nell'indice;
- i risultati sono tutti di tipo fuoco perché i primi 10 documenti ritornati sono tutti di tipo fuoco ma ampliando i risultati, quindi eliminando il vincolo ".paging(0, 10), si hanno tutti i documenti che soddisfano tali criteri.

- Trovare la media dei valori di Attacco per tutti i tipi

In [36]:
req = aggregations.AggregateRequest().group_by("@type", reducers.avg("@att").alias("avg_att")).limit(0, 3)
r.ft("pokemon").aggregate(req).rows

[['type', '["Bug","Poison"]', 'avg_att', '60.9090909091'],
 ['type', '["Electric","Flying"]', 'avg_att', '93.3333333333'],
 ['type', '["Normal","Flying"]', 'avg_att', '76.0769230769']]

Questa è una query di aggregazione, questo permette di raggruppare tutti i valori in base ad un determinato campo (in questo caso "Type") per poi poter eseguire un'operazione su tutti i valori che hanno il campo uguale (in questo caso la media sull'attacco 'reducers.avg("@att")').

### Analisi delle tempistiche delle Query
Per l'analisi delle tempistiche su Redis, tramite le modalità da noi eseguite, ci sono due opzioni:

1) Utilizzando la libreria Time importandola semplicemente aggiungendo: import time;
2) Utilizzando l'interfaccia di Redis su browser dove vi è la possibilità di scrivere query e uno dei dati mostrati è il tempo di esecuzione delle query.

Tutti i tempi forniti sono in secondi.
Il tempo risultante è uguale sia che si utilizzi la prima opzione sia che si utilizzi la seconda.

In [27]:
def avg_time(times):
    print(sum(times) / len(times))

#### Prima Query

In [28]:
times = []

overhead_start = time.time()
overhead = time.time() - overhead_start
for i in range(10):
    start = time.time()
    r.ft("pokemon").search("Pika*")
    end = time.time()
    times.append(end - start - overhead)

avg_time(times)

0.0013433933258056641


#### Seconda Query

In [29]:
times = []

overhead_start = time.time()
overhead = time.time() - overhead_start
for i in range(10):
    start = time.time()
    query = Query("@type:Fire")
    r.ft("pokemon").search(query)
    end = time.time()
    times.append(end - start - overhead)

avg_time(times)

0.001299285888671875


#### Terza Query

In [30]:
times = []

overhead_start = time.time()
overhead = time.time() - overhead_start
for i in range(10):
    start = time.time()
    query = Query("@att:[50 100]").return_fields("english", "att").paging(0, 10)
    r.ft("pokemon").search(query)
    end = time.time()
    times.append(end - start - overhead)

avg_time(times)

0.0012349605560302735


#### Quarta Query

In [31]:
times = []

overhead_start = time.time()
overhead = time.time() - overhead_start
for i in range(10):
    start = time.time()
    query = Query("@def:[0 90] | @type:Fire").return_fields("english", "type", "att").paging(0, 10)
    r.ft("pokemon").search(query)
    end = time.time()
    times.append(end - start - overhead)

avg_time(times)

0.0012802839279174804


#### Quinta Query

In [35]:
times = []

overhead_start = time.time()
overhead = time.time() - overhead_start
for i in range(10):
    start = time.time()
    req = aggregations.AggregateRequest().group_by("@type", reducers.avg("@att"))
    r.ft("pokemon").aggregate(req)
    end = time.time()
    times.append(end - start - overhead)

avg_time(times)

0.005329585075378418


I valori decimali mostrati al di sotto alle sezioni di codice sono il tempo di esecuzione delle query. Le query sono quelle sopra spiegate e riproposte con l'aggiunta delle funzioni relative al tempo.

Redis è un DBMS ad alte prestazioni come si può denotare dalle tempistiche di esecuzione dove sono tutte nell'ordine di millesimi di secondo:

- La query meno onerosa è la prima dove si cercano i pokemon che contengono "Pika" nel nome con un tempi di esecuzione di 0.0017 secondi;

- La query più onerosa è la quinta dove vengono eseguite le aggregazioni dove si ha un tempo di esecuzione di 0.0074 secondi all'interno di un database contenente 809 elementi;

- Tutte le altre query hanno un tempo di esecuzione simile compreso tra 0.0024 e 0.0026 secondi.

A differenza di ElasticSearch su Redis non viene effettuato uno scoring per determinare i risultati, quindi se si ha un OR in una query, chiunque rispetti i criteri verrà portato nei risultati.
Inoltre i risultati non sono in ordine in quanto redis non garantisce che i documenti siano salvati nell'ordine corretto in memoria, dunque è necessario eseguire un ordinamento.

### Possibili Miglioramenti
Così come per i molti db non relazionali, Redis permette una scalabilità di tipo orizzontale, creando varie repliche all'interno del cluster del DB.

Questa tipologia di scaling è molto onerosa e comporta nel tempo all'acquisto di macchine sempre più potenti per sopperire alla necessità di aumentare il numero di repliche.

Seguendo questo problema sono stati creati e resi disponibili molte varianti (chi più chi meno) del DBMS da noi scelto, uno di questi progetti è [Cachegrand](https://github.com/danielealbano/cachegrand), uno dei più veloci in-memory key-value database al momento disponibili sul mercato, creata e studiata dal basso, sfruttando al massimo le capacità hardware attuali.

Compatibilità con Redis è uno dei punti di forza, ma il maggiore (come si vedrà nei benchmark qui sotto) sono le performance.
Alcune features principali (anche elencate sul repo di github) sono:
- Support al Protocollo Redis
- Una Hashtable capace di valutare fino a 2.1 Miliardi di record al secondo (CPU in uso per il benchmark AMD EPYC 7502)
- Un FFMA o Fast Fixed Memory Allocator che permette di allocare e liberare la memoria in O(1)
- Scalabilità verticale, 2x CPU sono circa 2x richieste al secondo

<div>
<img src="https://raw.githubusercontent.com/cachegrand/cachegrand-benchmarks/main/images/latest-benchmarks-get-set.png" width="700"/>
</div>
Benchmark sul Get e Set
<div>
<img src="https://raw.githubusercontent.com/cachegrand/cachegrand-benchmarks/main/images/latest-benchmarks-get-set-pipelining.png" width="700">

Benchmark sul Get e Set in pipeline

# Contributors
Lorenzo Emanuele Avallone 866163\
Mattia Signorelli 852347